# Simulating time evolution

In this subsection, we detail the workflow for simulating the time evolution of a quantum system using common function "Evolve". Our focus is on implementing a Cross-resonance (CR) pulse in a system of three qubits. The code provided forms the basis of our discussion.

Initially, we set up the pulse parameters for our time evolution. We specify the length of the CR pulse and calculate the effective coupling strength $J_{eff}$ and the detuning between the dressed 0-1 frequencies of qubits 1 and 2. The driving amplitude of the pulse $\epsilon_d$ is computed based on the detuning and the effective coupling strength.

In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

fluxonium_1 = {
    "ec": 1.0 * 2 * np.pi,
    "ej": 4.0 * 2 * np.pi,
    "el": 0.9 * 2 * np.pi,
    "phiext": np.pi,
    "system_type": "fluxonium",
    "arguments": {"phi_max": 5 * np.pi},
}

fluxonium_2 = {
    "ec": 1.0 * 2 * np.pi,
    "ej": 4.0 * 2 * np.pi,
    "el": 1.0 * 2 * np.pi,
    "phiext": np.pi,
    "system_type": "fluxonium",
    "arguments": {"phi_max": 5 * np.pi},
}

fluxonium_3 = {
    "ec": 1.0 * 2 * np.pi,
    "ej": 4.0 * 2 * np.pi,
    "el": 1.1 * 2 * np.pi,
    "phiext": np.pi,
    "system_type": "fluxonium",
    "arguments": {"phi_max": 5 * np.pi},
}

coupling = {
    "capacitive_coupling": {"strength": 20.0e-3 * 2 * np.pi},
    "inductive_coupling": {"strength": -1.0 * 2e-3 * 2 * np.pi},
}


In [2]:
from supergrad.quantum_system.graph import BaseGraph


class MultipathThreeQubit(BaseGraph):
    def __init__(self):
        super().__init__()

        # nodes represent qubits
        self.add_node("q1", **fluxonium_1)
        self.add_node("q2", **fluxonium_2)
        self.add_node("q3", **fluxonium_3)
        # edges represent two-qubit interactions
        self.add_edge("q1", "q2", **coupling)
        self.add_edge("q2", "q3", **coupling)


In [3]:
from supergrad.common_functions import Spectrum


chain_3q = MultipathThreeQubit()
spec = Spectrum(chain_3q, share_params=True, unify_coupling=True)
params = spec.static_params
energy = spec.energy_tensor(params)
dressed_freq_q1 = (energy[1, 0, 0] - energy[0, 0, 0])
dressed_freq_q2 = (energy[0, 1, 0] - energy[0, 0, 0])


In [4]:
from supergrad.common_functions import Evolve
from supergrad.utils import tensor, compute_fidelity_with_1q_rotation_axis
from supergrad.utils.gates import cnot

length = 100.0
detuning = jnp.abs(dressed_freq_q1 - dressed_freq_q2)
j_eff = 0.01 * 2 * np.pi
tau_eps_drive = np.pi / 2.0 * detuning / j_eff

cr_pulse = {
    "pulse": {
        "amp": tau_eps_drive / length,
        "omega_d": dressed_freq_q2,
        "phase": 0.0,
        "length": length,
        "pulse_type": "cos",
        "operator_type": "phi_operator",
        "delay": 0.0,
    }
}


Hence we define the MultipathThreeQubitCR class, inheriting the MultipathThreeQubit model. This class includes the CR pulse configuration, thereby customizing the qubit model for our specific experiment.

In [5]:
class MultipathThreeQubitCR(MultipathThreeQubit):
    def __init__(self):
        super().__init__()

        self.add_node("q1", **cr_pulse)

cr_chain_3q = MultipathThreeQubitCR()


The Evolve class is then utilized to simulate the time evolution. We specify parameters such as the truncated dimension, parameter sharing, and compensation options. The unitary evolution of the CR pulse is computed using the eigen\_basis method of the Evolve class.

In [6]:
target_unitary = tensor(cnot(), jnp.eye(2))
evo = Evolve(cr_chain_3q, truncated_dim=3, share_params=True, unify_coupling=True, compensation_option='no_comp')
params = evo.pulse_params
cr_unitary = evo.eigen_basis(evo.static_params)
fideity, res_unitary = compute_fidelity_with_1q_rotation_axis(target_unitary, cr_unitary, compensation_option='arbit_single', n_qubit=3)
print(fideity)  # 0.98824


0.9882389768200743


The CR pulse, which contains an effective ZX term, can be utilized to implement a CNOT unitary. It is well-established that single-qubit gates generally exhibit lower error rates compared to two-qubit gates. Therefore, we can introduce virtual single-qubit rotations to simulate a CNOT unitary more accurately. For instance, we simulate a 3-qubit gate that is composed of a tensor product between a CNOT gate and an identity operation, and the possible compensations are calculated using a differentiable optimizer within the `compute_fidelity_with_1q_rotation_axis` function. Additionally, we compute the fidelity of the resultant unitary against the target unitary. The high fidelity(0.98824) indicates a successful implementation of the CNOT gate, demonstrating the effectiveness of our approach in simulating quantum gate operations in a multi-qubit system.

Furthermore, we conduct pulse parameter optimization to simulate device calibration. This optimization process utilizes automatic differentiation to obtain gradients. After optimization, the fidelity of the $CNOT\otimes I$ gate is significantly enhanced, reaching up to 99.99\%. Consider the following code as an example:

In [7]:
import haiku as hk
from supergrad.utils.optimize import scipy_minimize

def infidelity(params):
    params = hk.data_structures.merge(evo.static_params, params)
    cr_unitary = evo.eigen_basis(params)
    fideity, res_unitary = compute_fidelity_with_1q_rotation_axis(target_unitary, cr_unitary, compensation_option='arbit_single', n_qubit=3)
    return jnp.abs(1 - fideity)

scipy_minimize(infidelity, params, method='L-BFGS-B', logging=True)


/Users/allegro/anaconda3/envs/aqgrad/lib/python3.11/site-packages/jax/_src/lax/lax.py:2660: ComplexWarning: Casting complex values to real discards the imaginary part
  x_bar = _convert_element_type(x_bar, x.aval.dtype, x.aval.weak_type)


step: 0
parameters:
{ 'q1_pulse_cos': { 'amp': 0.13067653633650167,
                    'length': 100.0,
                    'omega_d': 3.6580304111499444,
                    'phase': 0.0}}
gradient:
{ 'q1_pulse_cos': { 'amp': -0.8874590856776485,
                    'length': -0.0013356914947876548,
                    'omega_d': -0.04429716346440153,
                    'phase': 1.089402151817024e-05}}
loss: 0.011761
step: 1
parameters:
{ 'q1_pulse_cos': { 'amp': 1.1294319972703935,
                    'length': 100.0015032007628,
                    'omega_d': 3.7078828884392716,
                    'phase': -1.226024236880252e-05}}
gradient:
{ 'q1_pulse_cos': { 'amp': 0.21603797553751458,
                    'length': 0.015263899413124081,
                    'omega_d': 16.561186395737916,
                    'phase': -5.798658440503154e-06}}
loss: 0.236806
step: 2
parameters:
{ 'q1_pulse_cos': { 'amp': 0.48488704961859685,
                    'length': 100.00053311299371,
       

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1.7077973705870164e-05
        x: q1_pulse_cos:     amp: 0.18574501113994388
                          length: 99.99939702795743
                         omega_d: 3.6816327791066814
                           phase: -8.951191617077884e-05
      nit: 24
      jac: q1_pulse_cos:     amp: -0.000575449599678444
                          length: 1.067118492499513e-05
                         omega_d: 0.0006838881048441983
                           phase: 7.776450585612178e-06
     nfev: 53
     njev: 53
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>